In [1]:
!pip3 install "transformers[torch]"
!pip3 install datasets
!pip3 install seqeval
!pip3 install scikit-learn
!pip3 install evaluate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\nicol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [1]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import torch
import lib
import numpy as np
import evaluate
import seqeval
from transformers import GenerationConfig

from collections import defaultdict

In [2]:
train_dataset = load_from_disk('full_valid_dataset_sentence-76975/train')
test_dataset = load_from_disk('full_valid_dataset_sentence-76975/test')
eval_dataset = load_from_disk('full_valid_dataset_sentence-76975/eval')
model_name = 'bert-large-uncased'

In [3]:
num_labels = len(lib.id2label)
max_length = 64

gen_config = GenerationConfig(max_length=max_length)

tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

new_tokens = ['[MATH]', '[DISPLAY]', '[ITALIC]']
num_added_tokens = tokenizer.add_tokens(new_tokens)

model.resize_token_embeddings(len(tokenizer))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30525, 1024)

In [4]:
def retokenize(row):
    tokens = ['[CLS]'] + row['tokens'][:max_length - 2] + ['[SEP]']
    annotations = ['O'] + row['spacy'][:max_length - 2] + ['O']
    remaining = max_length - (len(tokens))
    input_ids = tokenizer.convert_tokens_to_ids(tokens + ['[PAD]'] * remaining)
    
    row['input_ids'] = input_ids
    row['attention_mask'] = [1 if token_id != tokenizer.pad_token_id else 0 for token_id in input_ids]
    row['labels'] = [lib.label2id[label] for label in annotations] + [-100] * remaining
    return row
    
train_dataset = train_dataset.map(retokenize)
test_dataset = test_dataset.map(retokenize)
eval_dataset = eval_dataset.map(retokenize)

In [5]:
train_dataset

Dataset({
    features: ['file_name', 'title', 'tokens', 'math_index', 'nltk_annotate', 'spacy', 'is_valid', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 61580
})

In [13]:
metric = evaluate.load('seqeval')

get_correct_name_from_semi = defaultdict(str, {
    "DJ": "ADJ",
    "DP": "ADP",
    "DV": "ADV",
    "UX": "AUX",
    "ONJ": "CONJ",
    "CONJ": "CCONJ",
    "ET": "DET",
    "NTJ": "INTJ",
    "OUN": "NOUN",
    "UM": "NUM",
    "ART": "PART",
    "RON": "PRON",
    "ROPN": "PROPN",
    "UNCT": "PUNCT",
    "CONJ": "SCONJ",
    "YM": "SYM",
    "ERB": "VERB",
    "_": "X",
    "PACE": "SPACE"
})

def get_standard_dict():
    d = {}
    
    for key, value in get_correct_name_from_semi.items():
        d[value] = 0.0
    
    return d


def compute_metrics(p, all_metrics=False):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [lib.id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [lib.id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    return results

    res = {} #get_standard_dict()

    res['f1'] = results['overall_f1']
    res['accuracy'] = results['overall_accuracy']
    res['precision'] = results['overall_precision']
    res['recall'] = results['overall_recall']

    for key, value in results.items():
        if key not in ['overall_precision', 'overall_f1', 'overall_recall', 'overall_accuracy']:
            res[get_correct_name_from_semi[key]] = value['f1']

    return res



In [7]:
if torch.backends.mps.is_available():
    print("Using mps")
    mps_device = torch.device("mps")
    model.to(mps_device)

if torch.cuda.is_available():
    print("Using cuda")
    cuda_device = torch.device("cuda")
    model.to(cuda_device)

Using cuda


In [8]:


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=1900,
    weight_decay=0.013,
    label_smoothing_factor=0.009,
    learning_rate=0.0001,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("Starting training")
trainer.train()
print("Ending training")

Starting training


Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall,Part,Sconj,Adj,Adp,Adv,Verb,Det,Intj,Noun,Pron,Propn,Num,Punct,Aux,Sym
1,0.200400,0.183582,0.952831,0.963141,0.952590,0.953072,0.967295,0.979952,0.880320,0.988607,0.950524,0.959732,0.997120,0.529801,0.921254,0.976248,0.504295,0.913177,0.997211,0.993482,0.613333


In [ ]:
model.save_pretrained('./trained_bert_model_full_data_modified')

In [10]:
num_labels = len(lib.id2label)
max_length = 64

gen_config = GenerationConfig(max_length=max_length)

tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained("trained_bert_model_full_data_modified", num_labels=num_labels)

new_tokens = ['[MATH]', '[DISPLAY]', '[ITALIC]']
num_added_tokens = tokenizer.add_tokens(new_tokens)

model.resize_token_embeddings(len(tokenizer))

if torch.backends.mps.is_available():
    print("Using mps")
    mps_device = torch.device("mps")
    model.to(mps_device)

if torch.cuda.is_available():
    print("Using cuda")
    cuda_device = torch.device("cuda")
    model.to(cuda_device)




training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=20,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=1900,
    weight_decay=0.013,
    label_smoothing_factor=0.009,
    learning_rate=0.0001,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

Using cuda


In [ ]:
model.save_pretrained('./trained_bert_model_full_data_modified')

In [14]:
predictions = trainer.predict(eval_dataset)

model_predictions = predictions.predictions

true_label_ids = predictions.label_ids

p = (model_predictions, true_label_ids)
all_metrics = compute_metrics(p, True)

In [16]:
all_metrics

{'ART': {'precision': 0.9916820702402958,
  'recall': 0.9781221513217867,
  'f1': 0.9848554382744379,
  'number': 1097},
 'CONJ': {'precision': 0.9928168291431503,
  'recall': 0.9933264887063655,
  'f1': 0.9930715935334873,
  'number': 3896},
 'DJ': {'precision': 0.9427351500690695,
  'recall': 0.9537542878922627,
  'f1': 0.9482127068333965,
  'number': 7871},
 'DP': {'precision': 0.9954564666103127,
  'recall': 0.995772117112356,
  'f1': 0.9956142668428004,
  'number': 9461},
 'DV': {'precision': 0.9843071786310518,
  'recall': 0.980705256154358,
  'f1': 0.9825029161806367,
  'number': 3006},
 'ERB': {'precision': 0.9848256834712816,
  'recall': 0.9812570286142697,
  'f1': 0.9830381172936096,
  'number': 8003},
 'ET': {'precision': 0.9992404513888888,
  'recall': 0.9990236493816446,
  'f1': 0.9991320386242811,
  'number': 9218},
 'NTJ': {'precision': 0.8780487804878049,
  'recall': 0.8372093023255814,
  'f1': 0.8571428571428572,
  'number': 43},
 'OUN': {'precision': 0.963732807006723